<a href="https://colab.research.google.com/github/mixidota2/kaggle-indoor/blob/main/notebook/Indoor_001_baseline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Overview
- Baselineを構築するためのnotebook
- とりあえずデータ読んで最低限のsubをするだけを目的とする

In [ ]:
!nvidia-smi

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



In [28]:
from googleapiclient.discovery import build
import io, os
from googleapiclient.http import MediaIoBaseDownload
from google.colab import auth


auth.authenticate_user()
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
drive_service = build('drive', 'v3')
results = drive_service.files().list(
        q="name = 'kaggle.json'", fields="files(id)").execute()
kaggle_api_key = results.get('files', [])

filename = "/root/.kaggle/kaggle.json"
os.makedirs(os.path.dirname(filename), exist_ok=True)

request = drive_service.files().get_media(fileId=kaggle_api_key[0]['id'])
fh = io.FileIO(filename, 'wb')
downloader = MediaIoBaseDownload(fh, request)
done = False
while done is False:
    status, done = downloader.next_chunk()
    print("Download %d%%." % int(status.progress() * 100))
os.chmod(filename, 600)

mv: cannot stat 'kaggle.json': No such file or directory
Download 100%.


In [48]:
!kaggle datasets download -d kokitanisaka/indoorunifiedwifids
!unzip indoorunifiedwifids.zip

 99% 457M/463M [00:05<00:00, 103MB/s] 
100% 463M/463M [00:05<00:00, 87.2MB/s]


In [21]:
!pip install kaggle

In [52]:
import os
import glob 
import pickle
import random

import pandas as pd
import numpy as np

import yaml
from tqdm import tqdm
from joblib import Parallel, delayed

import seaborn as sns
import matplotlib.pyplot as plt

import cv2

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import mean_squared_error
from sklearn.decomposition import TruncatedSVD

import torch
import torch.nn as nn 
import torch.nn.functional as F

tqdm.pandas(position=0, leave=True)

/usr/local/lib/python3.7/dist-packages/tqdm/std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [53]:
with open(f'train_all.pkl', 'rb') as f:
  data = pickle.load( f)

In [55]:
data.head()

,bssid_0,bssid_1,bssid_2,bssid_3,bssid_4,bssid_5,bssid_6,bssid_7,bssid_8,bssid_9,bssid_10,bssid_11,bssid_12,bssid_13,bssid_14,bssid_15,bssid_16,bssid_17,bssid_18,bssid_19,bssid_20,bssid_21,bssid_22,bssid_23,bssid_24,bssid_25,bssid_26,bssid_27,bssid_28,bssid_29,bssid_30,bssid_31,bssid_32,bssid_33,bssid_34,bssid_35,bssid_36,bssid_37,bssid_38,bssid_39,...,rssi_65,rssi_66,rssi_67,rssi_68,rssi_69,rssi_70,rssi_71,rssi_72,rssi_73,rssi_74,rssi_75,rssi_76,rssi_77,rssi_78,rssi_79,rssi_80,rssi_81,rssi_82,rssi_83,rssi_84,rssi_85,rssi_86,rssi_87,rssi_88,rssi_89,rssi_90,rssi_91,rssi_92,rssi_93,rssi_94,rssi_95,rssi_96,rssi_97,rssi_98,rssi_99,x,y,floor,path,site_id
0,db01605eac3f33540038bd9722aba25774871d43,965f254a2e8d05bbb40bd2413ff61de3ad6c4151,0b64e537cc3d1818ec46f94f8dc14043a98d0089,922e582c66016a2b9f64e38f89ebe82f66eefb24,dc4c46287575c45f3e32c022d868d047b485ed4c,93e20595eeef175d3aa3c3381f6a22ee792d48d9,b2b0ddbb5a2aadfc6ab2f388db584b6c280d3f82,8c936564ea4b4300576f53136505527eb5972c07,61c3aaf1a526f808c05952ea3f098e37354a674a,3f564032c7eebc173b38aee35225e323d4389faf,2ed2e7ff03b520f03de97026028d9aada89dd4ba,0f5daed11a61e0d6941a1a42ff428ca216d61003,e315aa306303cc8df91c60dcbd80145ef496b649,1f37bbb3f42125f665b83584d0376b21ec3eb43c,c8f3983a40e3c462cdfd155f3d0c77868a717ffd,4c83a7a1e51bfa8a5fa20e854ab3feec057c52c9,d090480c29bbcafe61fdf7bf491e23c606d3f181,599fa96d549ed870671d6bc1927aaa8bbaacca12,51782c2fabefa97e99dca895fd36f1a47e214610,db85cb1371261046f2b711ac8dcceeb06b7ca724,53a65f2ceb68d4fab8fe912e36b2ca5d410a61be,e57024faa1bfe6de5d71e1b779bff11e72e97e1d,71d97fb3d6f464d4c26fd061732e58398d053b2e,add8b64585688391c11f2e0d1ca9039b70c802e5,ce28608c3d091ac0d25d84459ebad253edf83e1f,edbe427e90919b00d712ceb8c1ee8e92a54051e2,1bb0e992cff45a54d29e97f47a7d1281435a5e3b,172b8a530c6f1946a495576ea9e885874db2f288,1b76d774360c4cfaf89e75ec3a3add7c5201b885,ca86c5b074c5768e481e069b751bf22c6d95bd48,40d99a3e5214aa704f637b7d72631e69550ee256,46c934893439700099d03a6892ea934ecb2729d6,fe3211f90e4ab1f500e10fe175ae6142f4b13130,569878b7f574a738d49e365f45c49f44b81936f2,9f6279762e596eb60cd9a3870c1c305692d8e338,ae54f8552a572ddf81302b56c07f63c6321270be,3c89886dd08bb4a24384cf8bc0c6423e4804e6e9,5c10b343d767a30515e6015de25751a2883328f8,16374260af7d03b10f167358a4f6a70620e131f4,f26678bbbbd078e242638a0d1fb5ba2e61262f4c,...,-74,-75,-75,-75,-75,-75,-75,-76,-76,-76,-76,-76,-76,-77,-77,-77,-77,-77,-77,-78,-78,-78,-78,-78,-79,-79,-79,-79,-79,-79,-79,-79,-79,-79,-79,107.85044,161.89262,-1,5e1580adf4c3420006d520d4,5a0546857ecc773753327266
1,965f254a2e8d05bbb40bd2413ff61de3ad6c4151,db01605eac3f33540038bd9722aba25774871d43,1f37bbb3f42125f665b83584d0376b21ec3eb43c,922e582c66016a2b9f64e38f89ebe82f66eefb24,dc4c46287575c45f3e32c022d868d047b485ed4c,93e20595eeef175d3aa3c3381f6a22ee792d48d9,5c10b343d767a30515e6015de25751a2883328f8,3f564032c7eebc173b38aee35225e323d4389faf,46c934893439700099d03a6892ea934ecb2729d6,16374260af7d03b10f167358a4f6a70620e131f4,ae54f8552a572ddf81302b56c07f63c6321270be,3c89886dd08bb4a24384cf8bc0c6423e4804e6e9,ce9e2ebf59e424773fcdbd986f7633306bf03124,ce28608c3d091ac0d25d84459ebad253edf83e1f,2ed2e7ff03b520f03de97026028d9aada89dd4ba,51782c2fabefa97e99dca895fd36f1a47e214610,0f5daed11a61e0d6941a1a42ff428ca216d61003,8c936564ea4b4300576f53136505527eb5972c07,e315aa306303cc8df91c60dcbd80145ef496b649,599fa96d549ed870671d6bc1927aaa8bbaacca12,d090480c29bbcafe61fdf7bf491e23c606d3f181,c8f3983a40e3c462cdfd155f3d0c77868a717ffd,4c83a7a1e51bfa8a5fa20e854ab3feec057c52c9,0b64e537cc3d1818ec46f94f8dc14043a98d0089,db85cb1371261046f2b711ac8dcceeb06b7ca724,61c3aaf1a526f808c05952ea3f098e37354a674a,53a65f2ceb68d4fab8fe912e36b2ca5d410a61be,71d97fb3d6f464d4c26fd061732e58398d053b2e,b2b0ddbb5a2aadfc6ab2f388db584b6c280d3f82,0c5cda0386b9de9d4e8d5788f192af0dac40ce2c,b742b3fa0287399e647252d184ee8d58b67c05ae,b51107fb094a5127ad76ef502ac03632a8f0d67c,2aa08d092d0199c06d22684642ef1c79d9722adb,f26678bbbbd078e242638a0d1fb5ba2e61262f4c,a830915240b76b1da7286b938f5178b244d29b29,149c09a117b9851201c75f97b4a7cc94b75fdcb4,b5ae5